In [31]:
!guardrails hub install hub://brainlogic/high_quality_translation

Installing hub://brainlogic/high_quality_translation...
[==  ] Fetching manifestst
[=   ] Downloading dependenciespendencies  Running command git clone --filter=blob:none --quiet https://github.com/BrainLogicHub/high_quality_translation_validator.git /private/var/folders/c8/jqt82fpx785dpwpp36ljkgm40000gn/T/pip-req-build-cmoa3yt3
[ ===] Downloading dependenciesWARNING: Target directory /Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/guardrails/hub/brainlogic/high_quality_translation/validator already exists. Specify --upgrade to force replacement.
[  ==] Downloading dependencies
[   =] Running post-install setuptall setup/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
[====] Running post-install setup/Users/zaydsimjee/workspace/guardr

# Translate text with quality checks

**Note:**
To download this example as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/translation_with_quality_check.ipynb).

In this example, we will use Guardrails during the translation of a statement from another language to English. We will check whether the translated statement is likely of high quality.

## Objective

We want to translate a statement from different languages to English and ensure that the translated statement accurately reflects the original content.

### Setup

- Install the `unbabel-comet` from source:
  `pip install git+https://github.com/Unbabel/COMET`
- Please accept the model license from:
  https://huggingface.co/Unbabel/wmt22-cometkiwi-da
- Login into Huggingface Hub using:
  huggingface-cli login --token $HUGGINGFACE_TOKEN


In [32]:
import openai
from pydantic import BaseModel, Field
from guardrails import Guard
from rich import print
from guardrails.hub import HighQualityTranslation

## Step 1: Define a Guard that uses the validator

This guard will use the HighQualityTranslation validator to validate some string outputs.


We define the prompt and the guard.

In [33]:
prompt = """
Translate the given statement into English:

${statement_to_be_translated}
"""

guard = Guard().use(HighQualityTranslation(on_fail="fix"))


Loading the model Unbabel/wmt22-cometkiwi-da...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.2.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
Encoder model frozen.
/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']



## Step 2: Wrap the LLM API call with `Guard`


First, let's try translating a statement that is relatively easy to translate.


In [34]:
statement = "Ich habe keine Ahnung, was ich hier schreiben soll."

res = guard(
    openai.chat.completions.create,
    prompt=prompt,
    prompt_params={"statement_to_be_translated": statement},
    metadata={"translation_source": statement},
    model="gpt-3.5-turbo",
    max_tokens=1024,
    temperature=0,
)

print(f"Raw LLM Output: {res.raw_llm_output}")
print(f"Validated Output: {res.validated_output}")

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.

/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/zaydsimjee/workspace/guardrails/.

Raw LLM Output: I have no idea what I should write here.

Validated Output: I have no idea what I should write here.

We can look at the logs to see the quality check results:


In [35]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Translate the given statement into English:                                                             │ │
    │ │                                                                                                         │ │
    │ │ Ich habe keine Ahnung, was ich hier schreiben soll.                                                     │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭───────────────────────────────────────────── Instructions ──────────────────────────────────────────────╮ │
    │ │ You are a helpful assistant, expressing yourself through a string.                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ I have no idea what I should write here.                                                                │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ 'I have no idea what I should write here.'                                                              │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The `guard` wrapper returns the raw LLM response, which is the translated statement and also the validated output. In this case, the translated statement was of a good quality (above the threshold of 0.5), so the validated output is the same as the raw LLM response.

#### Now, let's test with a really low quality translation, and see how Guardrails handles it.


In [36]:
# Parse the code snippet
statement = "अरे भाऊ, आज रात्री जोरदार पार्टी मारूया, जमून टाकूया आणि धमाल करूया!"

## Ideal translation from Marathi -> English:
#  "Hey bro, let's have a great party tonight and have fun!"

output = guard.parse(
    llm_output="It's such a beautiful day, I'm going to the beach.",  ## here, providing a really bad translation
    metadata={"translation_source": statement},
)

# Print the output
print(f"Raw LLM Output: {output.raw_llm_output}")
print(f"Validated Output: {output.validated_output}")

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.

/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning

Raw LLM Output: It's such a beautiful day, I'm going to the beach.

Validated Output:

In [37]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │ No prompt                                                                                               │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ It's such a beautiful day, I'm going to the beach.                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ ''                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see, the translation quality is really bad, and the `HighQualityTranslation` check failed as the translation quality was below the threshold. The validated response is an empty string.

## In this way, you can use Guardrails to ensure that the output of your LLM is of high quality.
